https://stepik.org/lesson/1576184/step/8

In [ ]:
import torch
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()

_global_var_data_x = torch.tensor(data.data, dtype=torch.float32)
_global_var_target = torch.tensor(data.target, dtype=torch.int64)

In [ ]:
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim


class ThreeLayerModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(30, 32, bias=False)
        self.layer2 = nn.Linear(32, 20, bias=False)
        self.out = nn.Linear(20, 1)
        self.bm_1 = nn.BatchNorm1d(32)
        self.bm_2 = nn.BatchNorm1d(20)

    def forward(self, x):
        x = self.layer1(x).relu()
        x = self.bm_1(x)
        x = self.layer2(x).relu()
        x = self.bm_2(x)
        x = self.out(x)
        return x

# Модель
model = ThreeLayerModel()
model.train()

# Датасет из глобальных переменных с данными
ds = data.TensorDataset(_global_var_data_x, _global_var_target.float())

# Разбивка на train и test
train_ds, test_ds = data.random_split(ds, [0.7, 0.3])

# Делаем объекты DataLoader, разбитые на батчи
train_data = data.DataLoader(train_ds, batch_size=16, shuffle=True)
test_data = data.DataLoader(test_ds, batch_size=len(test_ds), shuffle=False)

# Оптимизатор Adam и функция потерь BCEWithLogitsLoss
optimizer = optim.Adam(params=model.parameters(), weight_decay=0, lr=0.01)
loss_func = nn.BCEWithLogitsLoss()

for _e in range(5): # epochs = 5
    for x, y in train_data:
        pred = model(x).squeeze()
        loss = loss_func(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

model.eval()

# Берём из тестовой выборки предикторы и целевую переменную
x_test, y_test = next(iter(test_data))

# Качество
predict = model(x_test).squeeze()
Q = (torch.round(torch.sigmoid(predict)) == y_test).sum().item() / len(y_test)

# print('Q =', Q)

Q = 0.8176470588235294
